<a href="https://colab.research.google.com/github/96jonesa/CSE-517-Project/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
!pip3 install --quiet "tensorflow-hub>=0.7.0"
!pip3 install --quiet seaborn
!pip3 install --quiet pandas-market-calendars

In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from absl import logging
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns
import json
import itertools
import pandas as pd
import torch
import pandas_market_calendars as mcal
import datetime
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

#Layers

In [3]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.gru = nn.GRU(input_size, hidden_size, batch_first=self.batch_first)

    def forward(self, input):
        output, hn = self.gru(input)
        return output, hn

In [4]:
# attention weights are softmax(u^T tanh(W input + b)) where W is learned parameter matrix, u is a learned parameter vector, and b is a learned offset

class LinearAttention(nn.Module):
    def __init__(self, input_size, intermediate_size, weights_size):
        super(LinearAttention, self).__init__()
        self.input_size = input_size
        self.intermediate_size = intermediate_size
        self.weights_size = weights_size

        self.linear_1 = nn.Linear(self.input_size, self.intermediate_size, bias=True)
        self.linear_2 = nn.Linear(self.intermediate_size, self.weights_size, bias=False)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=2)

    def forward(self, input, mask=None):
        intermediate = self.tanh(self.linear_1(input))
        pre_attention = self.linear_2(intermediate)
        if mask is not None:
            zero_vec = -9e15*torch.ones_like(pre_attention)
            pre_attention = torch.where(mask > 0, pre_attention, zero_vec)
        attention_weights = self.softmax(pre_attention)
        attention_weights = attention_weights.permute(0, 2, 1)
        output_features = torch.bmm(attention_weights, input)

        return output_features

In [5]:
# output is ReLU(left^T W right + b) where W is a learned paramater matrix
# and b is a learned bias

class Blend(nn.Module):
    def __init__(self, left_size, right_size, output_size):
        super(Blend, self).__init__()
        self.left_size = left_size
        self.right_size = right_size
        self.output_size = output_size

        self.bilinear = nn.Bilinear(self.left_size, self.right_size, output_size, bias=True)
        self.relu = nn.ReLU()
    
    def forward(self, left, right):
        output = self.relu(self.bilinear(left, right))

        return output

In [6]:
# https://github.com/Diego999/pyGAT/blob/master/layers.py

class SGAT(nn.Module):
    def __init__(self, input_size, output_size, leakyrelu_slope=0.01):
        super(SGAT, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.leakyrelu_slope = leakyrelu_slope
        
        self.W = nn.Parameter(torch.empty(size=(input_size, output_size)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*output_size, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        self.leakyrelu = nn.LeakyReLU(self.leakyrelu_slope)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        h_prime = torch.matmul(attention, Wh)

        return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        N = Wh.size()[0] # number of nodes
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)
        Wh_repeated_alternating = Wh.repeat(N, 1)

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)

        return all_combinations_matrix.view(N, N, 2 * self.output_size)

In [7]:
class MANSF(nn.Module):
    def __init__(self, T, gru_hidden_size, attn_inter_size, use_embed_size,
                 blend_size, gat_1_inter_size, gat_2_inter_size, leakyrelu_slope, elu_alpha, U):
        super(MANSF, self).__init__()
        self.T = T
        self.gru_hidden_size = gru_hidden_size
        self.attn_inter_size = attn_inter_size
        self.use_embed_size = use_embed_size
        self.blend_size = blend_size
        self.gat_1_inter_size = gat_1_inter_size
        self.gat_2_inter_size = gat_2_inter_size
        self.leakyrelu_slope = leakyrelu_slope
        self.elu_alpha = elu_alpha
        self.U = U

        self.gru_p = GRU(3, gru_hidden_size, batch_first=True)
        self.gru_m = GRU(use_embed_size, gru_hidden_size, batch_first=True)
        self.gru_s = GRU(gru_hidden_size, gru_hidden_size, batch_first=True)
        self.attn_p = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.attn_m = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.attn_s = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.blend = Blend(gru_hidden_size, gru_hidden_size, blend_size)
        self.mgat_1 = nn.ModuleList([SGAT(blend_size, gat_1_inter_size, leakyrelu_slope=leakyrelu_slope) for u in range(U)])
        self.mgat_2 = nn.ModuleList([SGAT(U * gat_1_inter_size, gat_2_inter_size, leakyrelu_slope=leakyrelu_slope) for u in range(U)])
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU(elu_alpha)
        self.final_linear = nn.Linear(U * gat_2_inter_size, 1, bias=True)

    # p is price data tensor of shape (num_stocks, T, 3), for the day under consideration
    # m is smi data list of tensors of shape (num_stocks, K, use_embed_size) of length T,
    #       where K is the number of tweets for the given stock on the day under consideration
    # neighorhoods is a list of adjacency lists, where each stock is indexed with the same
    #       indices they have in p and m
    def forward(self, p, m, m_mask, neighborhoods):
        ## price encoding
        h_p, _ = self.gru_p(p)
        q = self.attn_p(h_p)

        ## smi encoding (day level)
        r = torch.zeros(p.shape[0], 0, self.gru_hidden_size)
        r = r.to(device)
        for t in range(self.T):
            h_m, _ = self.gru_m(m[t])
            r_t = self.attn_m(h_m, m_mask[t])
            r = torch.cat((r, r_t), 1)

        ## smi encoding (aggregate)
        h_s, _ = self.gru_s(r)
        c = self.attn_s(h_s)

        ## blending
        x = self.blend(q, c)

        ## reshaping (eliminating superfluous dimension)
        x = x.view(x.shape[0], x.shape[2])

        ## first gat layer
        #  first head
        sgat = self.mgat_1[0]
        z = sgat(x, neighborhoods)
        z = self.elu(z)

        #  remaining heads
        for u in range(1, self.U):
            sgat = self.mgat_1[u]
            z_u = sgat(x, neighborhoods)
            z_u = self.elu(z_u)
            
            z = torch.cat((z, z_u), 1)
        
        ## second gat layer
        #  first head
        sgat = self.mgat_2[0]
        new_z = sgat(z, neighborhoods)
        new_z = self.sigmoid(new_z)

        #  remaining heads
        for u in range(1, self.U):
            sgat = self.mgat_2[u]
            new_z_u = sgat(z, neighborhoods)
            new_z_u = self.sigmoid(new_z_u)
            
            new_z = torch.cat((new_z, new_z_u), 1)
        
        ## final layer
        y = self.sigmoid(self.final_linear(new_z))

        ## return result
        return y

#Data Processing

In [8]:
#!wget https://github.com/yumoxu/stocknet-dataset/archive/master.zip

In [9]:
#!unzip master.zip

In [10]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [11]:
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
stocknet_dataset_filepath = './stocknet-dataset-master'
train_start_date = '2014-01-01'
train_end_date = '2015-07-31'
val_start_date = '2015-08-01'
val_end_date = '2015-09-30'
test_start_date = '2015-10-01'
test_end_date = '2016-01-01'

In [13]:
def prep_dataset(dataset_filepath, start_date, end_date):
    cache = {}
    calendar = mcal.get_calendar('NYSE')
    def next_trading_day(start_day=None, SAFE_DELTA = 4):
        """Returns the next/previous trading date separated by a certain number of 
        trading days.
        """
        if start_day is None:
            start_day = datetime.datetime.utcnow().date()
        if start_day in cache:
            return cache[start_day]
        start = pd.to_datetime(start_day)
        end = start + np.timedelta64(SAFE_DELTA, 'D')
        business_days = calendar.valid_days(start_date=start, end_date=end)
        next_day = business_days[1].date()
        next_day = next_day.strftime("%Y-%m-%d")
        cache[start_day] = next_day
        return next_day
    
    raw_prices_filepath = stocknet_dataset_filepath + '/price/raw'
    preprocessed_tweets_filepath = stocknet_dataset_filepath + '/tweet/preprocessed'

    company_to_price_df = {}
    company_to_tweets = {}

    for filename in os.listdir(raw_prices_filepath):
        with open(raw_prices_filepath + '/' + filename) as file:
            company_name = filename.split('.')[0]
            
            # Not enough data for GMRE
            if company_name == 'GMRE':
                continue
            df = pd.read_csv(file)
            df.columns = ['date', 'open', 'high', 'low', 'close', 'adjust_close', 'volume']
            mask = (df['date'] >= start_date) & (df['date'] <= end_date)
            df = df.loc[mask]
            company_to_price_df[company_name] = df.dropna()

    for filename in tqdm(os.listdir(preprocessed_tweets_filepath)):
        company_name = filename.split('.')[0]
        dates_to_tweets = {}
        for tweet_filename in os.listdir(preprocessed_tweets_filepath + '/' + filename):
            if tweet_filename < start_date or tweet_filename > end_date:
                continue
            with open(preprocessed_tweets_filepath + '/' + filename + '/' + tweet_filename) as file:
                list_of_tweets = []
                for line in file:
                    tweet_json = json.loads(line)
                    list_of_tweets.append(tweet_json)
                date_idx = next_trading_day(tweet_filename)
                if date_idx not in dates_to_tweets:
                    dates_to_tweets[date_idx] = list_of_tweets
                else:
                    dates_to_tweets[date_idx] += list_of_tweets
        company_to_tweets[company_name] = dates_to_tweets
    
    # Reduce logging output.
    logging.set_verbosity(logging.ERROR)
    tf.get_logger().setLevel(logging.ERROR)
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    # Import the Universal Sentence Encoder's TF Hub module
    def embed_useT(module):
        with tf.Graph().as_default():
            sentences = tf.placeholder(tf.string)
            embed = hub.Module(module)
            embeddings = embed(sentences)
            session = tf.train.MonitoredSession()
        return lambda x: session.run(embeddings, {sentences: x})
    embed_fn = embed_useT(module_url)

    # Generate embeddings
    for company in tqdm(company_to_tweets.keys()):
        for date in company_to_tweets[company].keys():
            messages = []
            for j in range(len(company_to_tweets[company][date])):
                messages.append(' '.join(company_to_tweets[company][date][j]['text']))
                message_embeddings = embed_fn(messages)
            for k in range(len(company_to_tweets[company][date])):
                company_to_tweets[company][date][k]['embedding'] = list(message_embeddings[k])
    
    # Create date mapping
    date_universe = set()
    for company in company_to_price_df.keys():
        date_universe = date_universe.union(set(company_to_price_df[company].date))
    for company in company_to_tweets.keys():
        date_universe = date_universe.union(set(company_to_tweets[company].keys()))
    date_universe = sorted(list(date_universe))
    index_to_date = {i-5:d for i,d in enumerate(date_universe)}
    date_to_index = {d:i-5 for i,d in enumerate(date_universe)}

    # Calculate dimensions for tensor
    n_stocks = len(company_to_tweets.keys())
    n_days = len(date_universe)
    max_tweets = 0
    for c,d in itertools.product(company_to_tweets.keys(), date_universe):
        if d in company_to_tweets[c]:
            max_tweets = max(max_tweets, len(company_to_tweets[c][d]))
    # Create index mapping for stocks alphabetically
    company_to_index = {c:i for i,c in enumerate(sorted(list(company_to_tweets.keys())))}

    return company_to_price_df, company_to_tweets, date_universe, n_days, n_stocks, max_tweets

In [14]:
train_company_to_price_df, train_company_to_tweets, train_date_universe, train_n_days, train_n_stocks, train_max_tweets = prep_dataset(stocknet_dataset_filepath, train_start_date, train_end_date)
val_company_to_price_df, val_company_to_tweets, val_date_universe, val_n_days, val_n_stocks, val_max_tweets = prep_dataset(stocknet_dataset_filepath, val_start_date, val_end_date)
test_company_to_price_df, test_company_to_tweets, test_date_universe, test_n_days, test_n_stocks, test_max_tweets = prep_dataset(stocknet_dataset_filepath, test_start_date, test_end_date)

#Dataset and DataLoader

In [92]:
class StockDataset(Dataset):
    """Price dataset"""

    def __init__(self, company_to_price_df, company_to_tweets, date_universe, n_days, n_stocks, max_tweets):
        # Initialize class members
        self.n_stocks = n_stocks
        self.n_days = n_days
        self.max_tweets = max_tweets
        self.window = 6
        window = self.window

        # Build maps
        self.company_to_index = {c:i for i,c in enumerate(sorted(list(company_to_tweets.keys())))}
        self.date_to_index = {d:i for i,d in enumerate(date_universe)}
        self.index_to_date = {i:d for i,d in enumerate(date_universe)}

        # Store data
        self.company_to_price_df = company_to_price_df
        self.company_to_tweets = company_to_tweets
        
        # Get price data tensor: n_stocks, n_days, 3
        self.price_data = np.zeros((n_stocks, n_days, 3))
        for company in company_to_price_df.keys():
            df = company_to_price_df[company]
            df.reset_index(inplace=True, drop=True)
            # Look up specific rows in DF
            for index, row in df.iterrows():
                # Grab row with particular date
                if index != 0:
                    d_index = self.date_to_index[row['date']]
                    c_index = self.company_to_index[company]
                    self.price_data[c_index, d_index, 0] = row['high'] / prev_close
                    self.price_data[c_index, d_index, 1] = row['low'] / prev_close
                    self.price_data[c_index, d_index, 2] = row['close'] / prev_close
                prev_close = row['close']
                
        # Which stocks are usable for these dates, shape n_days n_stocks
        self.usable_stocks = torch.ones((self.n_days-7, self.n_stocks))
    
        # Labels of shape n_days, n_stocks
        self.labels = torch.zeros((self.n_days-7, self.n_stocks))
        

        # Get labels
        for i in range(self.n_days-7):
            # Day after (for label)
            day_after = self.index_to_date[i + window + 1]
            # Current day
            current_day = self.index_to_date[i + window]
            for company in self.company_to_price_df.keys():
                df = self.company_to_price_df[company]

                # Grab row with particular date
                post_row = df.loc[df['date'] == day_after]
                row = df.loc[df['date'] == current_day]
                c_index = self.company_to_index[company]

                if (len(post_row['close']) > 0) and (len(row['close']) > 0):
                    close = np.zeros((1))
                    close[0] = post_row['close']
                    close[0] /= row['close']
                    if close >= 1.0055:
                        self.labels[i, c_index] = 1
                    elif close <= 0.995:
                        self.labels[i, c_index] = 0
                    else:
                        self.usable_stocks[i, c_index] = 0
                else:
                    self.usable_stocks[i, c_index] = 0

    def __len__(self):
        return self.n_days-7

    def __getitem__(self, idx):
        """
        gets a price tensor of shape (n_stocks, 6, 3)
        gets a smi tensor of shape (n_stocks, 6, K, 512)
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Size of sliding window
        window = self.window
        
        # Current day's usable stocks from price filter
        usable_stocks = self.usable_stocks[idx]
        
        # Labels from price day
        labels = self.labels[idx]

        # Dates that we need to look up
        dates_range = [self.index_to_date[i] for i in range(idx + 1, idx + window + 1)]

        # Day after (for label)
        day_after = self.index_to_date[idx + window + 1]

        # Current day
        current_day = self.index_to_date[idx + window]

        # Get price data tensor: n_stocks, window, 3
        price_data = self.price_data[:, idx+1:idx+window+1, :]

        # Extract tweets for specific window
        smi_data = np.zeros((self.n_stocks, window, self.max_tweets, 512))
        tweet_counts = np.zeros((self.n_stocks, window))
        for company in self.company_to_tweets.keys():

            # Look up tweets from specific days
            for date_idx, date in enumerate(dates_range):
                n_tweets = 0
                tweets = []
                c_index = self.company_to_index[company]
                if date in self.company_to_tweets[company]:
                    n_tweets = len(self.company_to_tweets[company][date])
                    tweets = [self.company_to_tweets[company][date][k]['embedding'] for k in range(n_tweets)]
                else:
                    usable_stocks[c_index] = 0
                tweet_counts[c_index, date_idx] = n_tweets
                if n_tweets == 0:
                    usable_stocks[c_index] = 0
                for i,embedding in enumerate(tweets): 
                    #stocks, day, lags, tweet, embedding
                    smi_data[c_index, date_idx, i, :] = embedding[:]

        usable_stocks = (usable_stocks == 1)

        m_mask = torch.zeros(6, self.n_stocks, self.max_tweets, 1)
        for t in range(6):
            for i in range(self.n_stocks):
                m_mask[t, i, 0:int(round(tweet_counts[i][t]+1)), 0] = 1

        price_output = price_data[usable_stocks,:,:]
        smi_output = smi_data[usable_stocks,:,:,:]
        tweet_count = tweet_counts[usable_stocks,:]
        m_mask = m_mask[:,usable_stocks,:,:]
        labels = labels[usable_stocks]
        
        # construct output
        return price_output, smi_output, tweet_count, usable_stocks, labels, m_mask

In [93]:
train_dataset = StockDataset(train_company_to_price_df, train_company_to_tweets, train_date_universe, train_n_days, train_n_stocks, train_max_tweets)
val_dataset = StockDataset(val_company_to_price_df, val_company_to_tweets, val_date_universe, val_n_days, val_n_stocks, val_max_tweets)
test_dataset = StockDataset(test_company_to_price_df, test_company_to_tweets, test_date_universe, test_n_days, test_n_stocks, test_max_tweets)

KeyboardInterrupt: 

In [71]:
train_dataloader = DataLoader(train_dataset, batch_size=1,
                        shuffle=True, num_workers=0)

val_dataloader = DataLoader(val_dataset, batch_size=1,
                        shuffle=False, num_workers=0)

test_dataloader = DataLoader(test_dataset, batch_size=1,
                        shuffle=False, num_workers=0)

#Separator

In [72]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#Training

In [73]:
mansf = MANSF(T=6,
              gru_hidden_size=64,
              attn_inter_size=32,
              use_embed_size=512,
              blend_size=32,
              gat_1_inter_size=32,
              gat_2_inter_size=32,
              leakyrelu_slope=0.01,
              elu_alpha=1.0,
              U=8)

In [74]:
mansf = mansf.to(device)

In [75]:
optimizer = optim.Adam(mansf.parameters(), lr=5e-4)
loss_fn = nn.BCELoss(reduction='mean')

In [76]:
train_acc_list = []
val_acc_list = []

In [78]:
for epoch in range(18):
    mansf.train()
    correct = 0.0
    total = 0.0
    running_loss = 0.0
    for price, smi, n_tweets, usable_stocks, labels, m_mask in tqdm(train_dataloader):
        price = price.type(torch.FloatTensor)
        smi = smi.type(torch.FloatTensor)
        print(price)
        print(price[:,])

        price = price.to(device)
        smi = smi.to(device)
        n_tweets = n_tweets.to(device)
        usable_stocks = usable_stocks.to(device)
        labels = labels.to(device)
        m_mask = m_mask.to(device)

        price = price.view(price.shape[1], price.shape[2], price.shape[3])
        smi = smi.view(smi.shape[1], smi.shape[2], smi.shape[3], smi.shape[4])
        n_tweets = n_tweets.view(n_tweets.shape[1], n_tweets.shape[2])
        usable_stocks = usable_stocks.view(usable_stocks.shape[1])
        m_mask = m_mask.view(m_mask.shape[1], m_mask.shape[2], m_mask.shape[3], m_mask.shape[4])

        smi = smi.permute(1, 0, 2, 3)

        m = []
        for t in range(6):
            m.append(smi[t])

        neighborhoods = torch.eye(87, 87)
        neighborhoods = neighborhoods.to(device)
        neighborhoods = neighborhoods[usable_stocks, :]
        neighborhoods = neighborhoods[:, usable_stocks]

        if price.shape[0] != 0:
            y = mansf(price, smi, m_mask, neighborhoods)
            loss = loss_fn(y.view(-1), labels.view(-1))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            correct += torch.sum(((y > 0.5).view(-1) == labels.view(-1))).item()
            total += len(y)
            running_loss = loss.item() * len(y)

    train_acc = correct / total
    train_acc_list.append(train_acc)

    mansf.eval()
    correct = 0.0
    total = 0.0
    for price, smi, n_tweets, usable_stocks, labels, m_mask in tqdm(val_dataloader):
        price = price.type(torch.FloatTensor)
        smi = smi.type(torch.FloatTensor)

        price = price.to(device)
        smi = smi.to(device)
        n_tweets = n_tweets.to(device)
        usable_stocks = usable_stocks.to(device)
        labels = labels.to(device)
        m_mask = m_mask.to(device)

        price = price.view(price.shape[1], price.shape[2], price.shape[3])
        smi = smi.view(smi.shape[1], smi.shape[2], smi.shape[3], smi.shape[4])
        n_tweets = n_tweets.view(n_tweets.shape[1], n_tweets.shape[2])
        usable_stocks = usable_stocks.view(usable_stocks.shape[1])
        m_mask = m_mask.view(m_mask.shape[1], m_mask.shape[2], m_mask.shape[3], m_mask.shape[4])

        smi = smi.permute(1, 0, 2, 3)

        m = []
        for t in range(6):
            m.append(smi[t])

        neighborhoods = torch.eye(87, 87)
        neighborhoods = neighborhoods.to(device)
        neighborhoods = neighborhoods[usable_stocks, :]
        neighborhoods = neighborhoods[:, usable_stocks]

        if price.shape[0] != 0:
            y = mansf(price, smi, m_mask, neighborhoods)
            correct += torch.sum((y > 0.5).view(-1) == labels.view(-1)).item()
            total += len(y)

    val_acc = correct / total
    val_acc_list.append(val_acc)

    print('epoch:', epoch, 'loss:', running_loss, 'train_acc:', train_acc, 'val_acc:', val_acc)

tensor([], size=(1, 0, 6, 3))
tensor([[[[1.0673, 1.0520, 0.9915],
          [1.0781, 1.0563, 1.0148],
          [1.0799, 1.0621, 1.0118],
          [1.0656, 1.0556, 1.0002],
          [1.0659, 1.0578, 1.0026],
          [1.0686, 1.0586, 1.0016]],

         [[1.0053, 0.9757, 0.9918],
          [1.0125, 0.9877, 1.0085],
          [1.0044, 0.9869, 0.9968],
          [1.0259, 1.0000, 1.0149],
          [1.0158, 1.0011, 1.0127],
          [1.0116, 0.9934, 0.9997]],

         [[1.1410, 1.1283, 0.9931],
          [1.1426, 1.1331, 1.0036],
          [1.1368, 1.1162, 0.9817],
          [1.1480, 1.1323, 0.9983],
          [1.1435, 1.1346, 1.0006],
          [1.1572, 1.1379, 1.0125]],

         [[1.0025, 0.9711, 0.9779],
          [1.0092, 0.9895, 1.0017],
          [1.0265, 0.9922, 1.0205],
          [1.0166, 0.9826, 0.9890],
          [1.0331, 0.9947, 1.0330],
          [1.0164, 0.9985, 1.0005]],

         [[0.9985, 0.9825, 0.9873],
          [1.0035, 0.9913, 1.0013],
          [1.0176, 0.9941,

tensor([[[[1.0683, 1.0472, 0.9929],
          [1.0748, 1.0605, 1.0020],
          [1.0864, 1.0687, 1.0114],
          [1.0830, 1.0664, 1.0119],
          [1.0697, 1.0590, 1.0010],
          [1.0672, 1.0538, 0.9869]],

         [[1.1511, 1.1192, 1.0220],
          [1.1438, 1.1153, 1.0054],
          [1.1363, 1.1116, 1.0089],
          [1.1417, 1.1174, 1.0181],
          [1.1433, 1.1191, 1.0117],
          [1.1265, 1.1070, 1.0026]],

         [[1.0075, 0.9670, 1.0005],
          [1.0251, 0.9957, 1.0240],
          [1.0154, 0.9879, 1.0038],
          [1.0192, 0.9920, 1.0183],
          [1.0200, 0.9942, 0.9953],
          [1.0116, 0.9820, 0.9856]],

         [[1.1317, 1.1116, 0.9973],
          [1.1355, 1.1251, 1.0101],
          [1.1273, 1.1181, 0.9990],
          [1.1234, 1.1078, 0.9982],
          [1.1381, 1.1230, 1.0102],
          [1.1318, 1.1162, 0.9970]],

         [[1.0209, 0.9684, 1.0129],
          [1.0277, 0.9971, 1.0144],
          [1.0156, 0.9889, 0.9975],
          [1.0175, 0

tensor([[[[1.0507, 1.0283, 0.9818],
          [1.0713, 1.0432, 1.0181],
          [1.0565, 1.0327, 0.9931],
          [1.0600, 1.0424, 1.0110],
          [1.0684, 1.0543, 1.0167],
          [1.0660, 1.0429, 1.0113]],

         [[1.0104, 0.9912, 0.9915],
          [1.0249, 1.0031, 1.0215],
          [0.9993, 0.9798, 0.9902],
          [1.0091, 0.9896, 1.0075],
          [1.0031, 0.9901, 0.9962],
          [1.0109, 0.9860, 1.0087]],

         [[1.0790, 1.0683, 0.9918],
          [1.0896, 1.0779, 1.0058],
          [1.0796, 1.0612, 0.9969],
          [1.0957, 1.0821, 1.0139],
          [1.0864, 1.0705, 1.0055],
          [1.0874, 1.0604, 1.0079]],

         [[1.0049, 0.9785, 0.9882],
          [1.0111, 0.9944, 0.9991],
          [1.0000, 0.9860, 0.9993],
          [1.0408, 1.0010, 1.0261],
          [1.0131, 0.9942, 1.0060],
          [1.0172, 0.9858, 1.0011]],

         [[1.0609, 1.0419, 1.0203],
          [1.0431, 1.0231, 0.9988],
          [1.0405, 1.0270, 1.0000],
          [1.0450, 1

tensor([[[[1.0736, 1.0634, 1.0040],
          [1.0683, 1.0472, 0.9929],
          [1.0748, 1.0605, 1.0020],
          [1.0864, 1.0687, 1.0114],
          [1.0830, 1.0664, 1.0119],
          [1.0697, 1.0590, 1.0010]],

         [[1.0281, 0.9986, 1.0134],
          [1.0075, 0.9670, 1.0005],
          [1.0251, 0.9957, 1.0240],
          [1.0154, 0.9879, 1.0038],
          [1.0192, 0.9920, 1.0183],
          [1.0200, 0.9942, 0.9953]],

         [[1.0706, 1.0523, 1.0436],
          [1.0401, 1.0225, 1.0134],
          [1.0297, 1.0077, 0.9973],
          [1.0287, 1.0191, 1.0008],
          [1.0231, 1.0136, 0.9921],
          [1.0383, 1.0234, 1.0038]],

         [[1.0314, 0.9883, 1.0085],
          [1.0209, 0.9684, 1.0129],
          [1.0277, 0.9971, 1.0144],
          [1.0156, 0.9889, 0.9975],
          [1.0175, 0.9946, 1.0164],
          [1.0255, 1.0019, 1.0195]],

         [[1.1724, 1.1577, 1.0143],
          [1.1703, 1.1511, 1.0135],
          [1.1721, 1.1604, 1.0127],
          [1.1737, 1

tensor([[[[1.0192, 1.0006, 1.0181],
          [1.0084, 0.9937, 1.0061],
          [1.0093, 0.9942, 0.9977],
          [1.0069, 0.9934, 0.9992],
          [1.0143, 0.9970, 1.0072],
          [1.0017, 0.9910, 0.9957]],

         [[1.0405, 1.0104, 1.0387],
          [1.0269, 0.9978, 1.0088],
          [1.0064, 0.9769, 0.9819],
          [1.0134, 0.9910, 0.9965],
          [1.0101, 0.9911, 1.0085],
          [1.0042, 0.9901, 0.9961]],

         [[1.0362, 1.0245, 1.0072],
          [1.0286, 1.0184, 1.0046],
          [1.0331, 1.0194, 1.0058],
          [1.0325, 1.0219, 1.0044],
          [1.0289, 1.0169, 1.0013],
          [1.0349, 1.0227, 1.0091]],

         [[1.0900, 1.0786, 1.0058],
          [1.0868, 1.0743, 1.0029],
          [1.0970, 1.0767, 1.0148],
          [1.0886, 1.0755, 0.9979],
          [1.0825, 1.0678, 0.9939],
          [1.1031, 1.0856, 1.0158]],

         [[1.1008, 1.0912, 0.9972],
          [1.0970, 1.0850, 0.9947],
          [1.1135, 1.0978, 1.0074],
          [1.1010, 1

...

tensor([[[[1.0674, 1.0550, 1.0009],
          [1.0705, 1.0558, 1.0060],
          [1.0753, 1.0609, 1.0119],
          [1.0826, 1.0637, 1.0103],
          [1.0755, 1.0647, 1.0063],
          [1.0686, 1.0575, 0.9996]],

         [[1.0123, 0.9925, 1.0101],
          [1.0037, 0.9838, 0.9947],
          [1.0025, 0.9926, 0.9966],
          [1.0074, 0.9928, 1.0006],
          [1.0259, 1.0010, 1.0234],
          [1.0131, 0.9978, 1.0014]],

         [[1.1247, 1.1146, 0.9918],
          [1.1509, 1.1304, 1.0196],
          [1.1278, 1.1175, 0.9956],
          [1.1421, 1.1233, 1.0099],
          [1.1297, 1.1142, 0.9950],
          [1.1256, 1.0981, 0.9786]],

         [[1.1010, 1.0940, 0.9940],
          [1.1056, 1.0948, 0.9951],
          [1.1098, 1.1014, 1.0053],
          [1.1048, 1.0952, 0.9943],
          [1.1132, 1.1031, 1.0046],
          [1.1144, 1.1014, 1.0080]],

         [[1.1159, 1.1040, 0.9943],
          [1.1197, 1.1030, 0.9989],
          [1.1145, 1.1021, 0.9920],
          [1.1177, 1

tensor([[[[1.0781, 1.0563, 1.0148],
          [1.0799, 1.0621, 1.0118],
          [1.0656, 1.0556, 1.0002],
          [1.0659, 1.0578, 1.0026],
          [1.0686, 1.0586, 1.0016],
          [1.0755, 1.0610, 1.0113]],

         [[1.1009, 1.0868, 0.9970],
          [1.1073, 1.0928, 1.0041],
          [1.0988, 1.0777, 0.9865],
          [1.1124, 1.1006, 1.0106],
          [1.1094, 1.0993, 1.0067],
          [1.1029, 1.0975, 1.0043]],

         [[1.0461, 1.0339, 0.9973],
          [1.0605, 1.0374, 1.0110],
          [1.0482, 1.0332, 0.9905],
          [1.0598, 1.0446, 1.0055],
          [1.0547, 1.0425, 1.0068],
          [1.0539, 1.0425, 1.0007]],

         [[1.0264, 1.0152, 0.9983],
          [1.0306, 1.0150, 1.0071],
          [1.0269, 1.0144, 0.9953],
          [1.0348, 1.0260, 1.0064],
          [1.0295, 1.0212, 1.0062],
          [1.0286, 1.0218, 1.0032]],

         [[1.0091, 0.9762, 0.9897],
          [1.0070, 0.9890, 1.0004],
          [1.0144, 0.9960, 1.0018],
          [1.0132, 0

tensor([[[[1.0277, 0.9993, 1.0194],
          [1.0136, 0.9915, 0.9950],
          [1.0191, 0.9940, 1.0168],
          [1.0222, 1.0081, 1.0214],
          [1.0083, 0.9927, 1.0009],
          [0.9952, 0.9818, 0.9905]],

         [[1.0295, 1.0084, 1.0193],
          [1.0032, 0.9713, 0.9863],
          [1.0115, 0.9897, 1.0043],
          [1.0151, 0.9940, 0.9987],
          [1.0426, 1.0046, 1.0325],
          [1.0448, 0.9775, 1.0419]],

         [[1.0561, 1.0442, 1.0127],
          [1.0566, 1.0373, 1.0113],
          [1.0509, 1.0337, 1.0024],
          [1.0545, 1.0472, 1.0076],
          [1.0545, 1.0393, 1.0064],
          [1.0447, 1.0278, 0.9965]],

         [[1.0412, 1.0237, 1.0165],
          [1.0288, 1.0159, 1.0040],
          [1.0373, 1.0167, 1.0097],
          [1.0370, 1.0195, 1.0071],
          [1.0267, 1.0128, 0.9989],
          [1.0197, 1.0065, 0.9948]],

         [[1.1239, 1.1082, 1.0050],
          [1.1229, 1.0997, 0.9924],
          [1.1423, 1.1105, 1.0201],
          [1.1216, 1

tensor([[[[1.0721, 1.0583, 1.0132],
          [1.0670, 1.0530, 0.9998],
          [1.0712, 1.0579, 1.0132],
          [1.0608, 1.0524, 1.0027],
          [1.0646, 1.0502, 1.0049],
          [1.0722, 1.0572, 1.0120]],

         [[1.1321, 1.1189, 0.9956],
          [1.1326, 1.1236, 0.9991],
          [1.1375, 1.1259, 1.0041],
          [1.1377, 1.1276, 1.0077],
          [1.1456, 1.1312, 1.0089],
          [1.1338, 1.1225, 0.9968]],

         [[1.0116, 1.0000, 1.0052],
          [0.9985, 0.9834, 0.9917],
          [1.0195, 1.0030, 1.0129],
          [1.0083, 0.9989, 1.0072],
          [1.0047, 0.9825, 0.9910],
          [1.0148, 1.0045, 1.0130]],

         [[1.0030, 0.9951, 0.9984],
          [0.9965, 0.9877, 0.9909],
          [1.0218, 1.0054, 1.0214],
          [1.0054, 0.9932, 0.9998],
          [1.0082, 0.9928, 0.9980],
          [1.0192, 1.0044, 1.0151]]]])
tensor([[[[1.0744, 1.0535, 0.9889],
          [1.0688, 1.0530, 0.9888],
          [1.0791, 1.0707, 1.0039],
          [1.0790, 

KeyboardInterrupt: 

In [ ]:
mansf.eval()

price, smi, n_tweets, usable_stocks, labels, m_mask = next(iter(val_dataloader))

price = price.type(torch.FloatTensor)
smi = smi.type(torch.FloatTensor)

price = price.to(device)
smi = smi.to(device)
n_tweets = n_tweets.to(device)
usable_stocks = usable_stocks.to(device)
labels = labels.to(device)
m_mask = m_mask.to(device)

price = price.view(price.shape[1], price.shape[2], price.shape[3])
smi = smi.view(smi.shape[1], smi.shape[2], smi.shape[3], smi.shape[4])
n_tweets = n_tweets.view(n_tweets.shape[1], n_tweets.shape[2])
usable_stocks = usable_stocks.view(usable_stocks.shape[1])
m_mask = m_mask.view(m_mask.shape[1], m_mask.shape[2], m_mask.shape[3], m_mask.shape[4])

smi = smi.permute(1, 0, 2, 3)

m = []
for t in range(6):
    m.append(smi[t])

neighborhoods = torch.eye(87, 87)
neighborhoods = neighborhoods.to(device)
neighborhoods = neighborhoods[usable_stocks, :]
neighborhoods = neighborhoods[:, usable_stocks]

y = mansf(price, smi, m_mask, neighborhoods)

In [ ]:
print(y)

In [ ]:
print(labels)

#Figures

In [ ]:
def plot(X, Y, xlabel, ylabel, legend, title):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for i in range(len(Y)):
        ax.plot(X, Y[i], label=legend[i])

    plt.grid(color='0.95')
    plt.legend()
    ax.set(xlabel=xlabel, ylabel=ylabel, title=title)

In [ ]:
"""plot(range(18),
     [train_acc_list, val_acc_list],
     'epoch',
     'accuracy',
     ['training accuracy', 'validation accuracy'],
     'accuracy vs. epoch')"""